In [ ]:
import argparse, logging, time
import subprocess
import eumdac
import os
from datetime import date, datetime, timedelta
from glob2 import glob
from epct import api
import pandas as pd
import xarray
from tqdm import tqdm
from functools import partial
from zipfile import ZipFile
from dask.distributed import as_completed, wait
import psutil

In [ ]:
ROI = {"NSWE": [62,35,-15,28]}

INPUT_PATH = '/scratch/snx3000/kschuurm/DATA/EO:EUM:DAT:MSG:HRSEVIRI'
SAVE_PATH = '/scratch/snx3000/kschuurm/DATA/customized'
LOG_DIR = '/scratch/snx3000/kschuurm/DATA/customized/log'

chain = eumdac.tailor_models.Chain(
        id="Western_Europe",
        product="HRSEVIRI",
        format="netCDF4",
        roi=ROI,
        projection="geographic",
        # compression='internal'
    )



In [ ]:
results_zip = glob(os.path.join(INPUT_PATH, f"MSG3-SEVI-MSG15*.zip"))
results_nat = glob(os.path.join(INPUT_PATH, f"MSG3-SEVI-MSG15*.nat"))
results_zip_nat = []
for fn in results_zip:
    try:
        with ZipFile(fn, "r") as zf:
            zipped_files = [
                name for name in zf.namelist() if name.endswith(".nat")
            ]
            if zipped_files:
                results_zip_nat.append(fn)
    except Exception as e:
        print(f"Failed to read zip file {fn}. Removing file.")
        print(e)
        os.remove(fn)

In [ ]:
print(f"{chain.product} Start customisation.")

pbar = tqdm(total=len(results_zip_nat), desc=f"{chain.product}")  # Init pbar

tasks, client = api.submit_customisations(
    results_zip_nat,
    chain_config=chain.asdict(),
    target_dir=SAVE_PATH,
    log_dir=LOG_DIR,
)

i = 0
for t in as_completed(tasks.values()):
    pbar.update(n=1)
    i += 1
    if i % 100 == 0:
        print("RAM memory %:", psutil.virtual_memory()[2])
        print("CPU usage %:", psutil.cpu_percent(4))
    del t

In [ ]:
output_files = api.run_chain(
[results_nat[0]],
chain_config=chain.asdict(),
target_dir=SAVE_PATH
)

In [ ]:
api.read('chains', product='HRSEVIRI', expanded=True)

In [ ]:
api.config()['products']

In [6]:
import satpy
from glob2 import glob
from satpy import Scene

In [10]:
filenames = glob('/scratch/snx3000/kschuurm/DATA/EO:EUM:DAT:MSG:HRSEVIRI/*.nat')

sc = Scene(reader="seviri_l1b_native", filenames=filenames)

In [12]:
sc.load(['VIS006'])

In [15]:
sc['VIS006']

<xarray.DataArray 'reshape-f2ebf0dceb49f761d047c0fd3bb7fc15' (y: 3712, x: 3712)>
dask.array<mul, shape=(3712, 3712), dtype=float32, chunksize=(928, 3712), chunktype=numpy.ndarray>
Coordinates:
    acq_time  (y) datetime64[ns] NaT NaT NaT NaT NaT NaT ... NaT NaT NaT NaT NaT
    crs       object PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["unknown",...
  * y         (y) float64 -5.567e+06 -5.564e+06 ... 5.564e+06 5.567e+06
  * x         (x) float64 5.567e+06 5.564e+06 ... -5.564e+06 -5.567e+06
Attributes: (12/20)
    orbital_parameters:                     {'projection_longitude': 0.0, 'pr...
    time_parameters:                        {'nominal_start_time': datetime.d...
    sun_earth_distance_correction_applied:  True
    sun_earth_distance_correction_factor:   0.9833101683850087
    units:                                  %
    wavelength:                             0.635 µm (0.56-0.71 µm)
    ...                                     ...
    name:                                   VIS006
    resolution:                             3000.403165817
    calibration:                            reflectance
    modifiers:                              ()
    _satpy_id:                              DataID(name='VIS006', wavelength=...
    ancillary_variables:                    []